In [ ]:
from data.ARCADE.dataloader import ARCADEDataset
from torch.utils.data import DataLoader
pathJson = 'data/ARCADE/processed/dataset.json'
dataset = ARCADEDataset(json_path=pathJson, split='train', task='Unsupervised')
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
for images in dataloader:
    print(f"Batch shape: {images.shape}")
    break


Batch shape: torch.Size([1, 1, 256, 256])


In [ ]:
from zoo.backbones import sanityCheck
sanityCheck()

ImportError: cannot import name 'sanityCheck' from 'zoo.backbones' (d:\Collateral Coronary Vessels XAI\zoo\backbones.py)

In [6]:
from zoo.models import SegmentatorCoronare
from zoo.backbones import get_backbone
model = SegmentatorCoronare(backbone='swinv2_tiny_window16_256', pretrained=False, in_channels=1)
import torch

print("SWIN")
model = SegmentatorCoronare(backbone='swinv2_tiny_window16_256', pretrained=False, in_channels=1)
dummy = torch.randn(2, 1, 256, 256)
out = model(dummy)
print(out.shape)
    
    
print('ConvNext')
model = SegmentatorCoronare(backbone='convnext_tiny', pretrained=False, in_channels=1)
dummy = torch.randn(2, 1, 256, 256)
out = model(dummy)
print(out.shape)
    
print('ViT')
model = SegmentatorCoronare(backbone='vit_small_patch16_224', pretrained=False, in_channels=1)
dummy = torch.randn(2, 1, 256, 256)
out = model(dummy)
print(out.shape)

ModuleNotFoundError: No module named 'backbones'

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from skimage.filters import frangi, gaussian, threshold_otsu
from skimage import exposure, morphology, measure
from skimage.measure import label, regionprops, block_reduce
from skimage.morphology import disk, reconstruction

import numpy as np
from skimage.filters import frangi, gaussian, threshold_otsu
from skimage import exposure, morphology
from skimage.measure import label, regionprops, block_reduce
from skimage.morphology import disk, reconstruction

def get_vasomim_probability_map(img):
    mask_roi = img > 0.05 
    mask_roi = morphology.binary_erosion(mask_roi, disk(15))
    
    img_clahe = exposure.equalize_adapthist(img, clip_limit=0.02, kernel_size=(32, 32))
    img_smooth = gaussian(img_clahe, sigma=1.0)

    scales_config = [
        {'sigma': 3.5, 'thresh_factor': 0.60}, 
        {'sigma': 2.0, 'thresh_factor': 0.45}  
    ]
    
    accumulated_mask = np.zeros_like(img, dtype=bool)
    accumulated_prob = np.zeros_like(img, dtype=float)

    for i, config in enumerate(scales_config):
        f_scale = frangi(img_smooth, sigmas=[config['sigma']], black_ridges=True, beta=0.5, gamma=2)
        f_scale = f_scale * mask_roi 
        
        if f_scale.max() > 0:
            f_scale = f_scale / f_scale.max()
            
        thresh = threshold_otsu(f_scale) if f_scale.max() > 0 else 0.1
        candidates = f_scale > (thresh * config['thresh_factor'])
        
        valid_layer = np.zeros_like(candidates)
        
        if i == 0:
            labels = label(candidates)
            if labels.max() > 0:
                regions = regionprops(labels)
                regions.sort(key=lambda x: x.area, reverse=True)
                
                largest = regions[0]
                valid_layer[labels == largest.label] = 1
                
                if len(regions) > 1 and regions[1].area > largest.area * 0.2:
                     valid_layer[labels == regions[1].label] = 1
        else:
            prev_dilated = morphology.binary_dilation(accumulated_mask, disk(4))
            seed = candidates & prev_dilated
            
            if seed.sum() > 0:
                valid_layer = reconstruction(seed, candidates, method='dilation').astype(bool)
                
        accumulated_mask = accumulated_mask | valid_layer.astype(bool)
        accumulated_prob = np.maximum(accumulated_prob, f_scale * valid_layer)

    PATCH_SIZE = 16
    
    final_prob_dilated = morphology.dilation(accumulated_prob, disk(2))
    patch_density = block_reduce(final_prob_dilated, block_size=(PATCH_SIZE, PATCH_SIZE), func=np.mean)
    
    h_p, w_p = patch_density.shape
    y, x = np.ogrid[:h_p, :w_p]
    center_bias = np.exp(-((x - w_p/2)**2 + (y - h_p/2)**2) / (2 * (w_p/2.5)**2))
    
    vasomim_probs = patch_density * center_bias
    vasomim_probs = (vasomim_probs - vasomim_probs.min()) / (vasomim_probs.max() - vasomim_probs.min() + 1e-6)
    vasomim_probs = vasomim_probs + 0.02 
    vasomim_probs = vasomim_probs / vasomim_probs.sum()

    return vasomim_probs.flatten()

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from data.ARCADE.dataloader import ARCADEDataset
from torch.utils.data import DataLoader
from skimage.filters import frangi, gaussian, threshold_otsu
from skimage import exposure, morphology
from skimage.measure import label, regionprops
from skimage.morphology import disk, binary_closing, binary_opening, remove_small_objects, remove_small_holes

def get_vessel_segmentation(img):
    """
    Segmentare completă a vaselor coronariene folosind filtrul Frangi.
    Returnează mască binară la dimensiunea imaginii originale.
    """
    # 1. ROI mask
    mask_roi = img > 0.05 
    mask_roi = morphology.binary_erosion(mask_roi, disk(15))
    
    # 2. Pre-procesare îmbunătățită: Gaussian blur mai puternic + CLAHE
    img_blur = gaussian(img, sigma=1.5)  # Blur mai puternic pentru a conecta discontinuitățile
    img_clahe = exposure.equalize_adapthist(img_blur, clip_limit=0.03, kernel_size=(32, 32))
    
    # Smooth suplimentar după CLAHE
    img_smooth = gaussian(img_clahe, sigma=0.8)
    
    # 3. Aplică Frangi la scale multiple și mai dense
    scales_config = [
        {'sigma': 4.0, 'thresh_factor': 0.55},  # Vase foarte groase
        {'sigma': 3.0, 'thresh_factor': 0.58},  # Vase groase
        {'sigma': 2.2, 'thresh_factor': 0.48},  # Vase medii
        {'sigma': 1.5, 'thresh_factor': 0.42}   # Vase subțiri
    ]
    
    accumulated_mask = np.zeros_like(img, dtype=bool)
    
    for i, config in enumerate(scales_config):
        # Frangi filter cu parametri ajustați pentru continuitate
        f_scale = frangi(
            img_smooth, 
            sigmas=[config['sigma']], 
            black_ridges=True, 
            beta=0.5,   # Sensibilitate la structuri blob
            gamma=15    # Sensibilitate la contrast (mai mare = mai sensibil)
        )
        f_scale = f_scale * mask_roi 
        
        # Normalizare
        if f_scale.max() > 0:
            f_scale = f_scale / f_scale.max()
        
        # Threshold Otsu adaptat
        thresh = threshold_otsu(f_scale) if f_scale.max() > 0 else 0.1
        candidates = f_scale > (thresh * config['thresh_factor'])
        
        valid_layer = np.zeros_like(candidates)
        
        if i == 0:  # Prima scală: păstrează cele mai mari componente
            labels = label(candidates)
            if labels.max() > 0:
                regions = regionprops(labels)
                regions.sort(key=lambda x: x.area, reverse=True)
                
                # Păstrează cele mai mari 3 componente (pentru mai multe vase principale)
                for j, region in enumerate(regions[:3]):
                    if j == 0 or region.area > regions[0].area * 0.15:
                        valid_layer[labels == region.label] = 1
        else:  # Scale următoare: conectează cu anterioarele (dilatare mai agresivă)
            from skimage.morphology import reconstruction
            # Dilatare mai agresivă pentru a conecta mai bine
            prev_dilated = morphology.binary_dilation(accumulated_mask, disk(5))
            seed = candidates & prev_dilated
            
            if seed.sum() > 0:
                valid_layer = reconstruction(seed, candidates, method='dilation').astype(bool)
            else:
                # Dacă nu găsește seed, încearcă să adauge candidați din apropiere
                prev_dilated_more = morphology.binary_dilation(accumulated_mask, disk(8))
                seed = candidates & prev_dilated_more
                if seed.sum() > 0:
                    valid_layer = reconstruction(seed, candidates, method='dilation').astype(bool)
        
        accumulated_mask = accumulated_mask | valid_layer.astype(bool)
    
    # 4. Post-procesare mai agresivă pentru continuitate
    # Closing mai agresiv pentru a conecta întreruperile
    final_mask = binary_closing(accumulated_mask, disk(3))  # disk(3) în loc de disk(2)
    
    # Îndepărtare găuri mici
    final_mask = remove_small_holes(final_mask, area_threshold=300)
    
    # Opening mai ușor pentru a păstra detaliile
    final_mask = binary_opening(final_mask, disk(1))
    
    # Îndepărtare obiecte mici
    final_mask = remove_small_objects(final_mask, min_size=80)
    
    # Closing final pentru netezire
    final_mask = binary_closing(final_mask, disk(2))
    
    return final_mask.astype(np.uint8)


def generate_frangi_segmentations(num_patients=30, select_random=True, output_dir="VasoPlots"):
    """
    Generează segmentări Frangi pentru pacienții din train și le salvează.
    
    Args:
        num_patients: Număr de pacienți (None = toți)
        select_random: True = aleator, False = primii N
        output_dir: Folder unde se salvează imaginile
    """
    # Creează directorul de output
    os.makedirs(output_dir, exist_ok=True)
    
    # Încarcă datasetul
    pathJson = 'data/ARCADE/processed/dataset.json'
    dataset = ARCADEDataset(json_path=pathJson, split='train', task='Unsupervised')
    
    # Selectează indicii pacienților
    total_patients = len(dataset)
    if num_patients is None:
        num_patients = total_patients
    else:
        num_patients = min(num_patients, total_patients)
    
    if select_random:
        indices = random.sample(range(total_patients), num_patients)
    else:
        indices = list(range(num_patients))
    
    print(f"Procesare {num_patients} pacienți din {total_patients} totali...")
    
    # Procesează fiecare pacient
    for idx, patient_idx in enumerate(indices):
        # Obține imaginea
        img_tensor = dataset[patient_idx]
        img = img_tensor.squeeze().numpy()  # [256, 256]
        
        # Aplică segmentarea Frangi
        try:
            segmentation = get_vessel_segmentation(img)
            
        except Exception as e:
            print(f"Eroare la pacientul {patient_idx}: {e}")
            import traceback
            traceback.print_exc()
            continue
        
        # Creează figura cu 3 coloane
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Coloana 1: Imaginea originală
        axes[0].imshow(img, cmap='gray')
        axes[0].set_title('Imagine Originală')
        axes[0].axis('off')
        
        # Coloana 2: Segmentarea Frangi
        axes[1].imshow(segmentation, cmap='gray')
        axes[1].set_title('Segmentare Frangi')
        axes[1].axis('off')
        
        # Coloana 3: Overlay
        axes[2].imshow(img, cmap='gray')
        axes[2].imshow(segmentation, cmap='Reds', alpha=0.5)
        axes[2].set_title('Overlay')
        axes[2].axis('off')
        
        # Salvează figura
        output_path = os.path.join(output_dir, f'patient_{idx:03d}.png')
        plt.tight_layout()
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close()
        
        if (idx + 1) % 10 == 0:
            print(f"Procesat {idx + 1}/{num_patients} pacienți...")
    
    print(f"✓ Finalizat! Salvat în {output_dir}/")

# Rulează funcția
generate_frangi_segmentations(num_patients=30, select_random=True)

Procesare 30 pacienți din 1997 totali...


C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:20: FutureWarning: `binary_erosion` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.erosion` instead. Note the pixel shift by 1 for even-sized footprints (see docstring notes).
  mask_roi = morphology.binary_erosion(mask_roi, disk(15))
C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:63: FutureWarning: `binary_dilation` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.dilation` instead. Note the lack of mirroring for non-symmetric footprints (see docstring notes).
  prev_dilated = morphology.binary_dilation(accumulated_mask, disk(4))
C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:72: FutureWarning: `binary_closing` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.closing` instead.
  final_mask = binary_closing(accumulated_mask, disk(2))
C:\Users\alexb\AppDat

Procesat 10/30 pacienți...


C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:20: FutureWarning: `binary_erosion` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.erosion` instead. Note the pixel shift by 1 for even-sized footprints (see docstring notes).
  mask_roi = morphology.binary_erosion(mask_roi, disk(15))
C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:63: FutureWarning: `binary_dilation` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.dilation` instead. Note the lack of mirroring for non-symmetric footprints (see docstring notes).
  prev_dilated = morphology.binary_dilation(accumulated_mask, disk(4))
C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:72: FutureWarning: `binary_closing` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.closing` instead.
  final_mask = binary_closing(accumulated_mask, disk(2))
C:\Users\alexb\AppDat

Procesat 20/30 pacienți...


C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:20: FutureWarning: `binary_erosion` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.erosion` instead. Note the pixel shift by 1 for even-sized footprints (see docstring notes).
  mask_roi = morphology.binary_erosion(mask_roi, disk(15))
C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:63: FutureWarning: `binary_dilation` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.dilation` instead. Note the lack of mirroring for non-symmetric footprints (see docstring notes).
  prev_dilated = morphology.binary_dilation(accumulated_mask, disk(4))
C:\Users\alexb\AppData\Local\Temp\ipykernel_15796\2367773794.py:72: FutureWarning: `binary_closing` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.closing` instead.
  final_mask = binary_closing(accumulated_mask, disk(2))
C:\Users\alexb\AppDat

Procesat 30/30 pacienți...
✓ Finalizat! Salvat în VasoPlots/


In [ ]:
import timm

print(timm.list_models(pretrained=False))

AttributeError: module 'torch' has no attribute '_utils'